In [1]:
import json
import os
import warnings

import numpy as np
import pandas as pd


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
def save_object(obj, name, out_dir='out'):
    file_name = name if name.endswith('.json') else (name + '.json')
    file_path = os.path.join(out_dir, file_name)
    with open(file_path, 'w') as f:
        json.dump(obj, f)

def load_object(name, in_dir='out'):
    file_name = name if name.endswith('.json') else (name + '.json')
    file_path = os.path.join(in_dir, file_name)
    with open(file_path, 'r') as f:
        return json.load(f)

In [3]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [4]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [5]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca [Inne]',
    '23:59:00_motywacje'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'trip_start_time',
    'source', 
    'destination',
    'what_inne',
    'source-destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [6]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:00:00,dom,Inne,SKLEP,dom-Inne
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:40:00,Inne,dom,0,Inne-dom
2,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Kobieta,0,0,0,0,-
3,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Mężczyzna,0,0,0,0,-
4,LOS_67-67-401,0.0,2018-04-26,61 i więcej (emeryci kobiety),Kobieta,0,0,0,0,-


In [7]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)
sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']
data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')
data_filtered['trip_start_time'] = data_filtered['trip_start_time'].astype('str')
# data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
# data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
# data_filtered['what_inne'] = data_filtered['what_inne'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
# data_filtered['source-destination'] = data_filtered['source-destination'].str.replace('Inne', 'inne', regex=True).replace('szkoła', 'szkola', regex=True)

In [8]:
# data_filtered = data_filtered[
#     data_filtered['source-destination'] != '-'
# ]

In [11]:
data_filtered[
    data_filtered['trip_start_time'] == '1899-12-30 00:00:00'
]

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id
11117,LOS_39_39_1412,1.0,2018-10-12,25-44,K,1899-12-30 00:00:00,inne,dom,0,inne-dom,25-44_K,LOS_39_39_14122018-10-121.0


In [13]:
data_filtered = data_filtered[
    data_filtered['trip_start_time'] != '1899-12-30 00:00:00'
]

data_filtered = data_filtered.sort_values(by=['person_id', 'trip_start_time'])

data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id
4054,LOS_100-100-1305,1.0,2018-06-14,61-x,K,13:05:00,dom,Inne,Pętle,dom-Inne,61-x_K,LOS_100-100-13052018-06-141.0
4055,LOS_100-100-1305,1.0,2018-06-14,61-x,K,14:30:00,Inne,dom,0,Inne-dom,61-x_K,LOS_100-100-13052018-06-141.0
7939,LOS_100-100-164,1.0,2018-06-22,45-60,K,0,0,0,0,-,45-60_K,LOS_100-100-1642018-06-221.0
7911,LOS_100-100-164,1.0,2018-06-22,25-44,K,17:00:00,dom,praca,0,dom-praca,25-44_K,LOS_100-100-1642018-06-221.0
7928,LOS_100-100-2281,1.0,2018-06-22,25-44,K,08:00:00,dom,inne,inne,dom-inne,25-44_K,LOS_100-100-22812018-06-221.0


In [14]:
data_filtered[
    (data_filtered['trip_start_time'] == '0')
    & (data_filtered['source'] == 0)
    & (data_filtered['destination'] == 0)
    & (data_filtered['source-destination'] == '-')
].shape[0]

1643

In [15]:
data_filtered['travel_occured'] = np.where(data_filtered['source'] == 0, 0, 1)

data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id,travel_occured
4054,LOS_100-100-1305,1.0,2018-06-14,61-x,K,13:05:00,dom,Inne,Pętle,dom-Inne,61-x_K,LOS_100-100-13052018-06-141.0,1
4055,LOS_100-100-1305,1.0,2018-06-14,61-x,K,14:30:00,Inne,dom,0,Inne-dom,61-x_K,LOS_100-100-13052018-06-141.0,1
7939,LOS_100-100-164,1.0,2018-06-22,45-60,K,0,0,0,0,-,45-60_K,LOS_100-100-1642018-06-221.0,0
7911,LOS_100-100-164,1.0,2018-06-22,25-44,K,17:00:00,dom,praca,0,dom-praca,25-44_K,LOS_100-100-1642018-06-221.0,1
7928,LOS_100-100-2281,1.0,2018-06-22,25-44,K,08:00:00,dom,inne,inne,dom-inne,25-44_K,LOS_100-100-22812018-06-221.0,1


In [16]:
# sum travels per person and store it in 'no_travels' column
df_any_travel = data_filtered.groupby(by=['age_sex_comb', 'person_id'])['travel_occured'].sum().to_frame('no_travels').reset_index()

# counts distribution of no_travels grouped by age_sex_comb
df_any_travel = df_any_travel.groupby(by='age_sex_comb')['no_travels'].value_counts().to_frame('count').reset_index()

# any_travel column: 0 if no_travels==0 else 1
df_any_travel['any_travel'] = np.where(df_any_travel['no_travels'] == 0, 0, 1)

# get counts and probs for any_travel grouped by age_sex_comb
# df_any_travel = df_any_travel.groupby(by=['age_sex_comb', 'any_travel'])['count'].sum().to_frame('count').sort_values(by=['age_sex_comb', 'any_travel'], ascending=[True, True]).reset_index()
df_any_travel = df_any_travel.groupby(['age_sex_comb', 'any_travel'])['count'].sum().agg(
    {'count': lambda x: x, 'prob': lambda x: x / x.sum(level=0)}
).unstack(level=0).sort_values(by=['age_sex_comb', 'any_travel'], ascending=[True, True]).reset_index()

df_any_travel['count'] = df_any_travel['count'].astype('int32')

df_any_travel

,age_sex_comb,any_travel,count,prob
0,16-19_K,0,20,0.192308
1,16-19_K,1,84,0.807692
2,16-19_M,0,11,0.081481
3,16-19_M,1,124,0.918519
4,20-24_K,0,46,0.155932
5,20-24_K,1,249,0.844068
6,20-24_M,0,58,0.198630
7,20-24_M,1,234,0.801370
8,25-44_K,0,196,0.168240
9,25-44_K,1,969,0.831760


### Prepare to .json format

In [17]:
age_sex_comb_list = df_any_travel['age_sex_comb'].unique().tolist()

any_travel_dist = {}
for age_sex_comb in age_sex_comb_list:
    any_travel_dist[age_sex_comb] = {}

for idx, row in df_any_travel.iterrows():
    age_sex_comb = row['age_sex_comb']
    any_travel = row['any_travel']
    prob = row['prob']
    any_travel_dist[age_sex_comb][any_travel] = prob

any_travel_dist

{'16-19_K': {0: 0.19230769230769232, 1: 0.8076923076923077},
 '16-19_M': {0: 0.08148148148148149, 1: 0.9185185185185185},
 '20-24_K': {0: 0.15593220338983052, 1: 0.8440677966101695},
 '20-24_M': {0: 0.19863013698630136, 1: 0.8013698630136986},
 '25-44_K': {0: 0.1682403433476395, 1: 0.8317596566523605},
 '25-44_M': {0: 0.15977961432506887, 1: 0.8402203856749312},
 '45-60_K': {0: 0.20124223602484473, 1: 0.7987577639751553},
 '45-65_M': {0: 0.18851570964247022, 1: 0.8114842903575298},
 '6-15_K': {0: 0.18181818181818182, 1: 0.8181818181818182},
 '6-15_M': {0: 0.13793103448275862, 1: 0.8620689655172413},
 '61-x_K': {0: 0.3361847733105218, 1: 0.6638152266894782},
 '66-x_M': {0: 0.3891402714932127, 1: 0.6108597285067874}}

### Save do .json

In [18]:
out_dir = './'
file_name = 'any_travel_dist.json'

save_object(
    obj=any_travel_dist,
    name=file_name,
    out_dir=out_dir
)

___
___
___